# Titanic - Machine Learning from Disaster

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3136/logos/header.png" width=900>

- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

### Começando a entender a base

- **Podemos utilizar o ydata-profiling (antigo Pandas Profiling) para começar a entender essa base**
    - https://github.com/ydataai/ydata-profiling

In [1]:
# Importando o pandas
import pandas as pd

In [3]:
# Visualizando a base de treino

treino = pd.read_csv("train.csv")
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# Importando o ProfileReport do ydata-profiling
!pip install ydata-profiling

from ydata_profiling import ProfileReport

In [7]:
# Gerando a visualização
profile = ProfileReport(treino, title="Titanic_Treino")

In [8]:
# Criando o relatório - exportar para html
profile.to_file("Titanic_Treino.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

- **Agora vamos analisar essa base**

In [9]:
# Verificando as informações da base
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
# Verificando o tipo dos dados - dtypes
treino.dtypes.value_counts()

int64      5
object     5
float64    2
dtype: int64

In [11]:
# E os valores nulos - isnull
treino.isnull().sum().sort_values(ascending=False).head(5)

#sum - somar
#sort_values(ascending=False) - ordenar do maior ao menor
#head(5) - 5 primeiros valores

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

- **Agora vamos visualizar a base de teste**

In [12]:
# Visualizando a base de teste
teste = pd.read_csv("test.csv")
teste.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [13]:
# Verificando as informações da base
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [14]:
# Analisando os valores nulos - não são os mesmos que os valores nulos de treino
teste.isnull().sum().sort_values(ascending=False).head(5)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
dtype: int64

- Temos colunas que **possuem valores vazios na base de teste que não estão vazias na base de treino** (nesse caso, precisaríamos tratar essas colunas apenas na base de teste)

### Realizando os tratamentos iniciais dos dados

- **Podemos começar tratando os valores vazios**

In [15]:
# Verificando novamente valores vazios para a base de treino
treino.isnull().sum().sort_values(ascending=False).head(5)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
dtype: int64

In [16]:
# Retornando o shape da base
treino.shape

# há 687 linhas vazias do total de 891 linhas e 12 colunas

(891, 12)

In [17]:
# Visualizando novamente a base
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
# Verificando a cardinalidade dos dados - colunas com muitos valores diferentes
treino.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Pclass           3
Embarked         3
Survived         2
Sex              2
dtype: int64

- Colunas com **alta cardinalidade** como nome, número do ticket e até mesmo o código da cabine não vão ajudar o modelo pois podem torná-lo **menos generalizável**. Nesse caso, vamos **eliminar essas colunas**

In [19]:
# Eliminando para a base de treino
treino = treino.drop(["Name","Ticket","Cabin"],axis=1) 

# axis=1 - coluna
#coluna PassengerID não será eliminada porque ela é necessária para a análise

In [20]:
# Também eliminando para a base de teste
teste = teste.drop(["Name","Ticket","Cabin"],axis=1)

- Agora precisamos tratar a coluna **Age**, que possui valores vazios tanto no treino quanto no teste
    - Como essa coluna possui a idade dos passageiros, podemos utilizar a **média das idades** para substituir os valores vazios

In [21]:
# Verificando a média das idades para a base de treino
treino.Age.mean()

# optou-se por inserir a média das idades na coluna Age ao invés de excluí-la.

29.69911764705882

In [23]:
# Podemos agora selecionar as colunas com idade vazia e substituir por essa média
treino.loc[treino.Age.isnull(),"Age"] = treino.Age.mean()

In [24]:
# E fazendo o mesmo para a base de teste
teste.loc[teste.Age.isnull(),"Age"] = teste.Age.mean()

- Para a base de treino, ainda temos 2 valores vazios na coluna **Embarked**
    - Podemos verificar todos os valores nessa coluna e substituir os valores vazios pela **moda**

In [26]:
# Verificando os valores na coluna Embarked
treino.Embarked.value_counts()

# o valor 644 é o mais frequente, sendo a moda dessa coluna - estatística

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [27]:
# Selecionando a moda dessa coluna
treino.Embarked.mode()[0]

'S'

In [28]:
# Substituindo os valores vazios
treino.loc[treino.Embarked.isnull(),"Embarked"] = treino.Embarked.mode()[0]

- Para finalizar, para a base de teste ainda temos 1 valor vazio em **Fare**
    - Podemos utilizar a mesma lógica que fizemos para a coluna Age e substituir pela **média**

In [29]:
# Verificando a média da coluna Fare para a base de teste
teste.loc[teste.Fare.isnull(),"Fare"] = teste.Fare.mean()

- **Verificando novamente os valores vazios**

In [30]:
# Para a base de treino
treino.isnull().sum().sort_values(ascending=False).head(5)

# todos os valores foram tratados

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
dtype: int64

In [31]:
# Para a base de teste
teste.isnull().sum().sort_values(ascending=False).head(5)

# todos os valores foram tratados

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
dtype: int64

### Considerando apenas as colunas que não são de texto

In [33]:
# Verificando as colunas de que não são texto na base de treino -> object = texto
col_treino_nr = treino.columns[treino.dtypes !="object"]
col_treino_nr
# != diferente de

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [34]:
# Selecionando apenas os valores numéricos da base de treino
treino_nr = treino.loc[:,col_treino_nr]


In [36]:
# E para a base de teste
col_teste_nr = teste.columns[teste.dtypes !="object"]
col_teste_nr

# não há coluna Survived como na de treino

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [38]:
# e os valores numéricos da base de teste
teste_nr = treino.loc[:,col_teste_nr]

### Podemos agora selecionar um modelo para classificar esses dados
- Para começar, vamos testar entre:
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [39]:
# Importando o train_test_split
from sklearn.model_selection import train_test_split

In [42]:
# Separando a base de treino em X e y
x = treino_nr.drop(["PassengerId","Survived"], axis=1)
y = treino.Survived

# tira a coluna survived, mas ela continua na base porque precisa dela para submissão 
# tira o passagerId porque ele atrapalha na análise
# não trabalha com colunas de texto, só numéricas(treino_nr), porque os algoritmos não foram feitos para trabalhar com colunas de texto

In [43]:
# Separando em treino e validação
x_train, X_val ,y_train, y_val = train_test_split(x,y, test_size=0.33, random_state=42)


# para não confundir X_test com a base teste, foi usado X_val e y_val(validação)

- Para a **árvore de classificação**

In [44]:
# Fazendo a importação
from sklearn import tree

In [49]:
# Criando o classificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

# código da documentação do link da árvore de classificação
# _ac = árvore de classificação

In [50]:
# Fazendo o fit com os dados
clf_ac = clf_ac.fit(x_train,y_train)

In [51]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

In [53]:
# Importando
from sklearn.neighbors import KNeighborsClassifier

In [55]:
# Criando o classificador
clf_knn = KNeighborsClassifier(n_neighbors=3)
# código da documentação KNN

In [58]:
# Fazendo o fit com os dados
clf_knn = clf_knn.fit(x_train, y_train)

In [59]:
# Fazendo a previsão
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

In [60]:
# Importando
from sklearn.linear_model import LogisticRegression

In [61]:
# Criando o classificador
clf_rl = LogisticRegression(random_state=42)

In [62]:
# Fazendo o fit com os dados
clf_rl = clf_rl.fit(x_train, y_train)

In [64]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)

### E, para finalizar, vamos avaliar esses modelos
- Para fazer essa análise, podemos usar:
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

In [65]:
# Importando
from sklearn.metrics import accuracy_score


In [66]:
# Para a árvore
accuracy_score(y_val,y_pred_ac)

0.6169491525423729

In [67]:
# Para o knn
accuracy_score(y_val,y_pred_knn)

0.6542372881355932

In [68]:
# Para a regressão logística
accuracy_score(y_val,y_pred_rl)

# o melhor modelo entre os 3 por ter 0.72, ou seja 72% de acurácia

0.7254237288135593

- Avaliando a **matriz de confusão** - Verificar a distribuição de erros

In [69]:
# Importando
from sklearn.metrics import confusion_matrix

In [70]:
# Para a árvore
confusion_matrix(y_val,y_pred_ac)

array([[125,  50],
       [ 63,  57]], dtype=int64)

In [71]:
# Para o knn
confusion_matrix(y_val,y_pred_knn)

array([[133,  42],
       [ 60,  60]], dtype=int64)

In [72]:
# Para a regressão logística - está melhor que os outros
confusion_matrix(y_val,y_pred_rl)

array([[156,  19],
       [ 62,  58]], dtype=int64)

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [84]:
# Visualizando o X_train
x_train.head()

,Pclass,Age,SibSp,Parch,Fare
6,1,54.000000,0,0,51.8625
718,3,29.699118,0,0,15.5000
685,2,25.000000,1,2,41.5792
73,3,26.000000,1,0,14.4542
882,3,22.000000,0,0,10.5167


In [85]:
# Visualizando a base de teste (apenas com valores numéricos)
teste_nr.head()

# não deveria ter a coluna passengerId

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,1,3,22.0,1,0,7.2500
1,2,1,38.0,1,0,71.2833
2,3,3,26.0,0,0,7.9250
3,4,1,35.0,1,0,53.1000
4,5,3,35.0,0,0,8.0500


In [86]:
# Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id
X_teste = teste_nr.drop("PassengerId", axis=1)

In [87]:
# Utilizando a regressão logística na base de teste
y_pred =clf_rl.predict(X_teste)

In [ ]:
# Criando uma nova coluna com a previsão na base de teste

teste["Survived"] = y_pred

# pois precisa enviar os dados da forma passenderId e survived 

In [95]:
# Selecionando apenas a coluna de Id e Survived para fazer o envio
base_envio = teste[["PassengerId","Survived"]]

In [96]:
# Exportando para um csv
base_envio.to_csv("resultado1.csv", index=False)

### Conclusão

- A meta é 85% de acurácia.
- Nos próximos arquivos desse Github, há novos códigos para atingir essa meta
- O projeto mostra o aprendizado prático dos modelos de Machine Learning